In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from HW3_helper import *
RANDOM_STATE = 12345 #Do not change it!
np.random.seed(RANDOM_STATE) #Do not change it!

In [115]:
diabetes = pd.read_csv("datasets/diabetes.csv")
X = diabetes.drop(["Outcome"], axis=1)
y = diabetes["Outcome"]

In [116]:
def imputation(df, columns):
    from sklearn.impute import SimpleImputer
    columns = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]
    df_parts = df.copy()[columns]

    df_parts[df_parts==0] = np.nan
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')

    df_converted = pd.DataFrame(imp.fit_transform(df_parts), columns=columns)
    diabetes[columns] = df_converted
    diabetes.describe()
    return df_converted

In [117]:
diabetes = imputation(diabetes, ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"])

# Classification

In [127]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [119]:
rfc = RandomForestClassifier(random_state=RANDOM_STATE)
skf = StratifiedKFold(n_splits=5)

In [120]:
param_grid = [
    {'max_depth':[1,2,3,4,5,6,7,8,9, 10]},
    {'min_samples_split':[2,3,4,5,6,7,8,9,10]},
    ]

In [121]:
search = GridSearchCV(rfc, param_grid=param_grid, cv=5)
search.fit(X, y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [123]:
rfc_cross_val_score = np.mean(cross_val_score(rfc, X, y, cv=skf))
rfc_cross_val_score

0.7669977081741788

In [124]:
rf_best_classifier = search.best_estimator_
rf_best_classifier

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=9,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

## SVM with diverse kernels

In [126]:
scaler = StandardScaler()



scaled_data = scaler.fit_transform(X)
scaled_data = pd.DataFrame(X, columns=diabetes.columns)

In [122]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [ ]:
svm = SVC()
svc.fit(X, y)

